In [ ]:
!pip install transformers

In [ ]:
!pip install tensorflow

In [ ]:
!pip install datasets

In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np

In [ ]:
from google.colab import drive

# Precisamos montar o drive para que ele possa ler o dataset salvo no drive
# E ler também o modelo caso já exista um
# Posteriormente também iremos salvar o modelo no drive
drive.mount('/content/drive')
raiz_data_set = '/content/drive/MyDrive/pantanaldev/dataset/'

# Ele lê as bases de dados para treino, validação e teste
data_train = (pd.read_csv(raiz_data_set+ 'train.csv')).sample(frac=1.0)
data_validation = (pd.read_csv(raiz_data_set+ 'validation.csv')).sample(frac=1.0)
data_test = (pd.read_csv(raiz_data_set+ 'test.csv')).sample(frac=1.0)

Mounted at /content/drive


O código abaixa realiza a classsificação da partir do score realizado e coloca dentro de uma coluna chamada sentence

In [ ]:
# Função usada para converter score em sentimento
def get_sentiment(score):
    if score in [1, 2]:
        return 'negative'
    elif score == 3:
        return 'neutral'
    elif score in [4, 5]:
        return 'positive'
    
# Aplica a função lambda ao campo "score" e salva o resultado no novo campo "label"
data_train['label'] = data_train['Score'].apply(lambda x: get_sentiment(x))
data_train = data_train.rename(columns={'Text': 'sentence'})

data_test['label'] = data_test['Score'].apply(lambda x: get_sentiment(x))
data_test = data_test.rename(columns={'Text': 'sentence'})

data_validation['label'] = data_validation['Score'].apply(lambda x: get_sentiment(x))
data_validation = data_validation.rename(columns={'Text': 'sentence'})

O código abaixo remove todas as linhas da tabela com mais de 512 caracteres pois isso é o maxímo que o Bert consegue realizar

In [ ]:
max_length = 512
data_train = data_train.drop(data_train[data_train['sentence'].str.len() > max_length].index)
data_test = data_test.drop(data_test[data_test['sentence'].str.len() > max_length].index)
data_validation = data_validation.drop(data_validation[data_validation['sentence'].str.len() > max_length].index)

In [ ]:
porcentagens = {'negative': 10, 'positive': 10, 'neutral': 32}
num_rows = 1000

def definirPorcentagem(data):
# cria um DataFrame vazio para armazenar as amostras selecionadas
  amostras = pd.DataFrame()
  # para cada categoria, selecione uma amostra aleatória do número desejado de observações
  for categoria, porcentagem in porcentagens.items():
      # calcula o número de observações que você deseja selecionar
      num_observacoes = int(porcentagem / 100 * num_rows)
      
      # seleciona uma amostra aleatória de observações
      amostra = data.loc[data['label'] == categoria].sample(n=num_observacoes, replace=False)
      
      # adiciona a amostra selecionada ao DataFrame de amostras
      amostras = pd.concat([amostras, amostra])
  return amostras

# exibe o DataFrame 
data_train =  definirPorcentagem(data_train)

data_train = data_train.sample(frac=1.0)
data_validation  = data_validation.sample(frac=0.01)
data_test =  data_test.sample(frac=0.008)

print("Tamanho do treino:", len(data_train))
print("Tamanho da validação:", len(data_validation))
print("Tamanho do teste:", len(data_test))

In [ ]:
# Colocamos os dados dentro de um dicionário da biblioteca DatasetDict
from datasets import Dataset, DatasetDict
data = DatasetDict({
    'train': Dataset.from_pandas(data_train),
    'validation': Dataset.from_pandas(data_test),
    'test': Dataset.from_pandas(data_validation)
})

In [ ]:
from transformers import AutoTokenizer

# Carrega o tokenizador com o modelo do bert
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# É necessário colocar aqui a nossa base de dados
# Onde ele passa dataset["sentence"], é preciso passar a base de dados na coluna
# do texto a ser avaliado
tokenized_data_train = tokenizer(data['train']["sentence"], return_tensors="np",truncation=True, padding=True, max_length=512)
tokenized_data_test = tokenizer(data['test']['sentence'], return_tensors="np", truncation=True, padding=True, max_length=512)
tokenized_data_validation = tokenizer(data['validation']['sentence'], return_tensors="np", truncation=True, padding=True, max_length=512)

# Tokenizer returns a BatchEncoding, but we convert that to a dict for Keras
tokenized_data_train = dict(tokenized_data_train)
tokenized_data_test = dict(tokenized_data_test)
tokenized_data_validation = dict(tokenized_data_validation)

# É necessário também colocar aqui o nosso dataset na coluna que tem a rotulação
labels_train = np.array(data['train']["label"]) 
labels_test = np.array(data['test']["label"]) 
labels_validation =  np.array(data['validation']["label"]) 

In [ ]:
from transformers import TFAutoModelForSequenceClassification
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras.losses import SparseCategoricalCrossentropy

# Assim o modelo será pego do treino anterior
raiz_model = '/content/drive/MyDrive/pantanaldev/model/'
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased",num_labels=3)

# Inicializando a Loss com crossentropy
loss = SparseCategoricalCrossentropy(from_logits=True)

# Compilando o modelo com o otimizador adam e com metrica de precisão
model.compile(optimizer='adam',loss=loss, metrics=['accuracy'])

# Definindo um callback do Keras, para parar o treinamento em caso de overfitting
early_stop = EarlyStopping(monitor='val_loss', patience=2)


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Carrega os pesos do treino anterior
model.load_weights(raiz_model)

In [ ]:
# Codifica as rotulações para o uso da IA 
le = LabelEncoder()
encoded_labels_train = le.fit_transform(labels_train)
encoded_labels_test = le.fit_transform(labels_test)
encoded_labels_validation = le.fit_transform(labels_validation)


Mudar os parametros para determinar o menor loss

In [ ]:
# Colocando o modelo para treino
model.fit(tokenized_data_train,encoded_labels_train,epochs=8, batch_size=15,shuffle = True, validation_data=(tokenized_data_validation, encoded_labels_validation))

Salvar o código , o modelo

In [ ]:
# Salvando o modelo de 3 formas diferentes, por redundância
model.save_weights(raiz_model)
model.save(raiz_model + "Teste/")
tf.keras.models.save_model(model, raiz_model + "Teste2/")


In [ ]:
# Avaliação do modelo atual
test_scores = model.evaluate(tokenized_data_test, encoded_labels_test)


458/920 [=============>................] - ETA: 4:32:01 - loss: 1.3301

In [ ]:
# Predição do modelo atual com o teste
pred = model.predict(tokenized_data_test)

In [ ]:
# Abaixo pegamos as predições feitas no teste acima e as conferimos
probas = pred.logits
predicted_class = tf.argmax(probas, axis=-1)
predicted_class = predicted_class.numpy().tolist()

classes = ['negative','neutral','positive']

testing_labels = []

for l in labels_test:
  if l == 'positive':
    testing_labels.append(2)
  
  elif l == 'negative':
    testing_labels.append(0)

  elif l == 'neutral':
    testing_labels.append(1)

soma = 0

acerto = 0

for i in range(0,len(predicted_class)):
  print(f"Esperado {labels_test[i]} -> Retornado {classes[predicted_class[i]]}")

  if testing_labels[i] == predicted_class[i]:
    acerto += 1

  soma += 1


print("Taxa de acerto:", acerto/soma)
